In [1]:
from cProfile import label
from symbol import return_stmt

import numpy as np
import pandas as pd
import os
import cv2
import math
import random
import matplotlib.pyplot as plt
import shutil
from sklearn.preprocessing import QuantileTransformer
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('./CICIDS_final_baseline_smote.csv')  # was './CICIDS-7-number-packets-to-flows_FINAL.csv'

In [3]:
df.Label.value_counts()

0    34908
2    22979
1     8435
3     7752
4     5854
Name: Label, dtype: int64

In [5]:
from sklearn.utils import resample

dfs = []

target_size = 7000  # a compromise between smallest and largest

for cls in df['Label'].unique():
    class_df = df[df['Label'] == cls]
    if len(class_df) > target_size:
        class_df = resample(class_df, replace=False, n_samples=target_size, random_state=42)
    else:
        class_df = resample(class_df, replace=True, n_samples=target_size, random_state=42)
    dfs.append(class_df)

balanced_df = pd.concat(dfs)


In [6]:
balanced_df.Label.value_counts()

2    7000
0    7000
1    7000
3    7000
4    7000
Name: Label, dtype: int64

In [7]:
# Randomly shuffle the dataframe
balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)

In [8]:
df

,Total Length of Fwd Packets,Bwd IAT Min,Fwd IAT Total,Flow IAT Std,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Total Fwd Packets,Fwd Packet Length Min,...,Flow Bytes/s,Total Length of Bwd Packets,Fwd Packet Length Max,Flow IAT Mean,Bwd Packet Length Mean,Flow Packets/s,Bwd Header Length,Bwd Packet Length Min,Bwd Packet Length Max,Label
0,0.000000,0.000000,102.740240,0.000000,0.0,0.0,0.0,125.713213,119.842342,0.0,...,0.000000,0.000000,0.000000,10.720721,0.00000,245.427928,0.000000,0.0,0.000000,2
1,0.000000,0.000000,102.740240,0.000000,0.0,0.0,0.0,125.713213,119.842342,0.0,...,0.000000,0.000000,0.000000,10.720721,0.00000,245.427928,0.000000,0.0,0.000000,2
2,0.000000,0.000000,102.740240,0.000000,0.0,0.0,0.0,125.713213,119.842342,0.0,...,0.000000,0.000000,0.000000,10.720721,0.00000,245.427928,0.000000,0.0,0.000000,2
3,0.000000,0.000000,102.740240,0.000000,0.0,0.0,0.0,125.713213,119.842342,0.0,...,0.000000,0.000000,0.000000,10.720721,0.00000,245.427928,0.000000,0.0,0.000000,2
4,0.000000,0.000000,102.740240,0.000000,0.0,0.0,0.0,125.713213,119.842342,0.0,...,0.000000,0.000000,0.000000,10.720721,0.00000,245.427928,0.000000,0.0,0.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79923,243.003003,151.876877,186.846847,157.261465,0.0,0.0,0.0,251.936937,251.426426,0.0,...,126.082410,182.124625,243.768769,156.741160,109.75976,95.558763,249.639640,0.0,182.507508,4
79924,243.003003,175.105105,187.102102,156.336453,0.0,0.0,0.0,251.936937,251.426426,0.0,...,125.996103,182.124625,243.768769,156.098955,104.90991,96.064723,253.468468,0.0,182.507508,4
79925,243.003003,151.876877,187.102102,156.424167,0.0,0.0,0.0,251.936937,251.426426,0.0,...,125.942694,182.124625,243.768769,156.193052,104.90991,95.978379,253.468468,0.0,182.507508,4
79926,243.003003,175.105105,186.846847,154.890874,0.0,0.0,0.0,252.511261,251.426426,0.0,...,126.051043,182.124625,243.768769,156.002554,104.90991,96.153541,254.234234,0.0,182.507508,4


In [9]:
label_count = {i: 0 for i in range(0,5)}

In [10]:
count=0
ims = []

image_count = 0

label_count = {i: 0 for i in range(0,5)}

for i in range(0, len(balanced_df)):  
    count=count+1
    if count<=60: 
        im=balanced_df.iloc[i].drop('Label').values
        label_count[balanced_df.iloc[i]['Label']] += 1
        ims=np.append(ims,im)
    else:
        
        max_value = max(label_count.values())  # maximum value
        max_keys = [k for k, v in label_count.items() if v == max_value] # getting all keys containing the `maximum`
        
        this_label = max_keys[0]
        image_path = f"test_shuffled/{this_label}/"
        
        if not os.path.exists(image_path):
            os.makedirs(image_path)
        
        image_count = image_count + 1
        ims=np.array(ims).reshape(20,20,3)
        array = np.array(ims, dtype=np.uint8)
        new_image = Image.fromarray(array)
        new_image.save(image_path+str(i)+'.png')
        count=0
        ims = []
        label_count = {i: 0 for i in range(0,5)}

In [11]:
#resize the images 224*224 for better CNN training
def get_224(folder,dstdir):
    imgfilepaths=[]
    for root,dirs,imgs in os.walk(folder):
        for thisimg in imgs:
            thisimg_path=os.path.join(root,thisimg)
            imgfilepaths.append(thisimg_path)
    for thisimg_path in imgfilepaths:
        dir_name,filename=os.path.split(thisimg_path)
        dir_name=dir_name.replace(folder,dstdir)
        new_file_path=os.path.join(dir_name,filename)
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
        img=cv2.imread(thisimg_path)
        img=cv2.resize(img,(224,224))
        cv2.imwrite(new_file_path,img)
    print('Finish resizing'.format(folder=folder))

In [12]:
DATA_DIR2_224='./test_224_shuffled/'
get_224(folder='./test_shuffled/',dstdir=DATA_DIR2_224)

Finish resizing


In [13]:
label_count

{0: 8, 1: 8, 2: 10, 3: 6, 4: 15}